<a href="https://colab.research.google.com/github/MaxYarosh/ML-projects/blob/main/Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.metrics import log_loss, accuracy_score

In [17]:
# ---  Допоміжні функції ---
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

In [18]:
# --- Бінарна логістична регресія ---
class LogisticRegressionBinary:
    def __init__(self, lr=0.1, n_iter=1000):
        self.lr = lr
        self.n_iter = n_iter

    def fit(self, X, y):
        m, n = X.shape
        self.theta = np.zeros((n, 1))

        for _ in range(self.n_iter):
            y_pred = sigmoid(X @ self.theta)
            gradient = (1/m) * X.T @ (y_pred - y)
            self.theta -= self.lr * gradient

    def predict_proba(self, X):
        return sigmoid(X @ self.theta)

    def predict(self, X):
        return (self.predict_proba(X) >= 0.5).astype(int)

In [19]:
# ---Багатокласова логістична регресія (softmax) ---
class LogisticRegressionMulticlass:
    def __init__(self, lr=0.1, n_iter=1000):
        self.lr = lr
        self.n_iter = n_iter

    def fit(self, X, y):
        m, n = X.shape
        k = len(np.unique(y))
        Y = np.eye(k)[y]

        self.theta = np.zeros((n, k))

        for _ in range(self.n_iter):
            logits = X @ self.theta
            probs = softmax(logits)
            gradient = (1/m) * X.T @ (probs - Y)
            self.theta -= self.lr * gradient

    def predict_proba(self, X):
        return softmax(X @ self.theta)

    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)


In [20]:
# --- Тестування: бінарна класифікація ---
X_bin, y_bin = make_classification(
    n_samples=500,
    n_features=2,
    n_informative=2,
    n_redundant=0,
    n_classes=2,
    random_state=42
)
y_bin = y_bin.reshape(-1, 1)


In [21]:
#  bias
X_bin_b = np.c_[np.ones((X_bin.shape[0], 1)), X_bin]

model_bin = LogisticRegressionBinary(lr=0.1, n_iter=5000)
model_bin.fit(X_bin_b, y_bin)

y_pred_bin = model_bin.predict(X_bin_b)
print("Custom Binary Logistic Regression:")
print("Accuracy:", accuracy_score(y_bin, y_pred_bin))
print("Log loss:", log_loss(y_bin, model_bin.predict_proba(X_bin_b)))

Custom Binary Logistic Regression:
Accuracy: 0.892
Log loss: 0.2802393635568943


In [22]:
# sklearn
clf_bin = LogisticRegression()
clf_bin.fit(X_bin, y_bin.ravel())
print("\nsklearn LogisticRegression (binary):")
print("Accuracy:", clf_bin.score(X_bin, y_bin))


sklearn LogisticRegression (binary):
Accuracy: 0.89


In [23]:
#  Тестування: багатокласова класифікація ---
X_multi, y_multi = make_classification(n_samples=500, n_features=4, n_classes=3, n_informative=3, n_redundant=0, random_state=42)

#  bias
X_multi_b = np.c_[np.ones((X_multi.shape[0], 1)), X_multi]

model_multi = LogisticRegressionMulticlass(lr=0.1, n_iter=5000)
model_multi.fit(X_multi_b, y_multi)

y_pred_multi = model_multi.predict(X_multi_b)
print("\nCustom Multiclass Logistic Regression:")
print("Accuracy:", accuracy_score(y_multi, y_pred_multi))
print("Log loss:", log_loss(y_multi, model_multi.predict_proba(X_multi_b)))



Custom Multiclass Logistic Regression:
Accuracy: 0.598
Log loss: 0.7848571239664103


In [24]:

# sklearn
clf_multi = LogisticRegression(multi_class="multinomial", solver="lbfgs", max_iter=5000)
clf_multi.fit(X_multi, y_multi)
print("\nsklearn LogisticRegression (multiclass):")
print("Accuracy:", clf_multi.score(X_multi, y_multi))


sklearn LogisticRegression (multiclass):
Accuracy: 0.598


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
